# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [2]:
# Your code here
import sqlite3
import pandas as pd

conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [4]:
# Your code here
pd.read_sql("""SELECT firstName, lastName
FROM employees
JOIN offices
USING(officeCode)
WHERE offices.city == 'Boston'""", conn)

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [10]:
# Your code here
pd.read_sql("""SELECT offices.officeCode, offices.city, COUNT(employees.employeeNumber) as num_office_employees
FROM offices
LEFT JOIN employees
USING(officeCode)
GROUP BY offices.officeCode
HAVING num_office_employees = 0""", conn)

,officeCode,city,num_office_employees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [ ]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

In [13]:
"""
Question 1
How many employees are in San Francisco office?
"""


# Your code here

pd.read_sql("""SELECT offices.officeCode, offices.city, COUNT(employees.employeeNumber) as num_office_employees
FROM offices
LEFT JOIN employees
USING(officeCode)
WHERE offices.city == 'San Francisco'
GROUP BY offices.officeCode
""", conn)

,officeCode,city,num_office_employees
0,1,San Francisco,6


In [18]:
"""
Question 2
How many customers are in SF?
"""
pd.read_sql("""SELECT COUNT(customerNumber), city
FROM customers
GROUP BY city""", conn)
# Your code here

,COUNT(customerNumber),city
0,1,Aachen
1,1,Allentown
2,1,Amsterdam
3,1,Auckland
4,2,Auckland
...,...,...
91,1,Versailles
92,1,Warszawa
93,1,Wellington
94,1,White Plains


In [22]:
"""
Question 3
How many customers are there per office?
"""
q = """SELECT c.customerNumber, c.customerName, o.officeCode, o.city, COUNT(c.customerNumber) as customers_per_office
FROM customers as c
JOIN employees as e 
ON c.salesRepEmployeeNumber == e.employeeNumber
JOIN offices as o
USING(officeCode)
GROUP BY o.officeCode
"""
# Your code here
pd.read_sql(q,conn)

,customerNumber,customerName,officeCode,city,customers_per_office
0,112,Signal Gift Stores,1,San Francisco,12
1,157,Diecast Classics Inc.,2,Boston,12
2,131,Land of Toys Inc.,3,NYC,15
3,103,Atelier graphique,4,Paris,29
4,148,"Dragon Souveniers, Ltd.",5,Tokyo,5
5,114,"Australian Collectors, Co.",6,Sydney,10
6,121,Baane Mini Imports,7,London,17


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [28]:
# Your code here
pd.read_sql("""SELECT products.productName, employees.firstName, employees.lastName
FROM employees
JOIN customers
ON customers.salesRepEmployeeNumber = employees.employeeNumber
JOIN orders
USING(customerNumber)
JOIN orderdetails
USING(orderNumber)
JOIN products
USING(productCode)""", conn)

,productName,firstName,lastName
0,1958 Setra Bus,Leslie,Jennings
1,1940 Ford Pickup Truck,Leslie,Jennings
2,1939 Cadillac Limousine,Leslie,Jennings
3,1996 Peterbilt 379 Stake Bed with Outrigger,Leslie,Jennings
4,1968 Ford Mustang,Leslie,Jennings
...,...,...,...
2991,1954 Greyhound Scenicruiser,Martin,Gerard
2992,1950's Chicago Surface Lines Streetcar,Martin,Gerard
2993,Diamond T620 Semi-Skirted Tanker,Martin,Gerard
2994,1911 Ford Town Car,Martin,Gerard


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [39]:
# Your code here
# Your code here
pd.read_sql("""SELECT employees.firstName, employees.lastName, SUM(orderdetails.quantityOrdered) as products_sold
FROM employees
JOIN customers
ON customers.salesRepEmployeeNumber = employees.employeeNumber
JOIN orders
USING(customerNumber)
JOIN orderdetails
USING(orderNumber)
GROUP BY employees.lastName""", conn)

,firstName,lastName,products_sold
0,Loui,Bondur,6186
1,Larry,Bott,8205
2,Pamela,Castillo,9290
3,Julie,Firrelli,4227
4,Andy,Fixter,6246
5,Martin,Gerard,4180
6,Gerard,Hernandez,14231
7,Leslie,Jennings,11854
8,Barry,Jones,7486
9,Peter,Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [40]:
# Your code here
# Your code here
# Your code here
pd.read_sql("""SELECT products.productName, employees.firstName, employees.lastName, COUNT(orderdetails.quantityOrdered) as products_sold
FROM employees
JOIN customers
ON customers.salesRepEmployeeNumber = employees.employeeNumber
JOIN orders
USING(customerNumber)
JOIN orderdetails
USING(orderNumber)
JOIN products
USING(productCode)
GROUP BY employees.lastName
HAVING products_sold > 200""", conn)

,productName,firstName,lastName,products_sold
0,1972 Alfa Romeo GTA,Larry,Bott,236
1,1972 Alfa Romeo GTA,Pamela,Castillo,272
2,1965 Aston Martin DB5,Gerard,Hernandez,396
3,1958 Setra Bus,Leslie,Jennings,331
4,1952 Alpine Renault 1300,Barry,Jones,220
5,1969 Harley Davidson Ultimate Chopper,George,Vanauf,211


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!